In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from Quantum_Kernel import *

%matplotlib inline
# Importing standard Qiskit libraries and configuring account
from qiskit import QuantumCircuit , transpile, Aer ,IBMQ
from qiskit.visualization import *
from qiskit.tools.jupyter import *
from qiskit.providers.ibmq import least_busy
from qiskit.providers.aer import QasmSimulator

from qiskit import Aer, QuantumCircuit
from qiskit.opflow import Z, I, StateFn
from qiskit.utils import QuantumInstance
from qiskit.circuit import Parameter
from qiskit.circuit.library import RealAmplitudes, ZZFeatureMap
from qiskit.algorithms.optimizers import *

from qiskit.opflow import StateFn, PauliSumOp, AerPauliExpectation, ListOp, Gradient
from qiskit_machine_learning.neural_networks import OpflowQNN
import qiskit.quantum_info as qi
from scipy import optimize
from qiskit.opflow.gradients import Gradient
from typing import Union

from IPython.display import clear_output
from scipy.optimize import approx_fprime


# Loading your IBM Quantum account(s)
provider = IBMQ.load_account()

In [2]:
mat = pd.read_csv("malaria/Mat_train.csv")

In [3]:
mat_x = mat.iloc[:,1:]
mat_y = mat.iloc[:,0:1]

In [4]:
mat_xs = np.pi*(mat_x-np.min(mat_x))/(np.max(mat_x)-np.min(mat_x))
mat_ys = np.pi*(mat_y-np.min(mat_y))/(np.max(mat_y)-np.min(mat_y))

/Users/minhyeong-gyu/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.min(axis=None) will return a scalar min over the entire DataFrame. To retain the old behavior, use 'frame.min(axis=0)' or just 'frame.min()'
  return reduction(axis=axis, out=out, **passkwargs)
/Users/minhyeong-gyu/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)


In [5]:
mat_x

,vegetation,water,rain,x1,x2,x3,x4,x5,x6,x7,...,x230,x231,x232,x233,x234,x235,x236,x237,x238,x239
0,-0.630022,-0.408734,-1.181258,518.288114,516.131378,526.663793,389.888883,387.814227,397.949275,358.735868,...,643.968071,602.771985,574.781793,559.763559,557.584530,1505.303432,1358.585269,1227.914103,1112.904084,857.928251
1,-0.720276,-0.804297,0.396576,721.605987,767.077595,826.361951,570.774960,614.916849,672.588022,388.003650,...,421.803781,426.979751,444.561851,474.749176,517.857387,887.460400,798.548165,724.121922,663.758137,563.375079
2,-0.201062,2.803119,0.834390,1291.058632,1390.487211,1505.288112,1095.611910,1193.133977,1305.902450,721.467504,...,293.900222,340.677648,400.595716,474.287016,562.366285,398.728387,361.613465,336.946841,324.391011,358.409944
3,0.159907,0.470622,0.331857,2709.487059,2727.472131,2761.040264,2376.020316,2393.739566,2426.816374,2104.587896,...,13.337771,2.355428,-0.170820,-0.060039,4.746701,463.470392,354.919143,262.373866,185.307685,40.295648
4,1.405908,-0.738166,1.106541,1386.738654,1427.936322,1483.758201,1161.688087,1202.019697,1256.708933,923.428683,...,111.423191,110.021816,118.784956,138.066308,168.527576,524.574727,435.005529,359.937094,298.722329,191.336800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3495,-1.667931,0.307437,-1.977657,49.789170,60.189900,80.418326,20.409368,29.398388,47.572834,21.965937,...,1711.810551,1684.650540,1672.231760,1674.515756,1691.509634,2613.165991,2476.682988,2356.343459,2251.915438,2032.041201
3496,-0.331652,-0.587832,-0.720589,253.174445,238.368831,235.008625,172.004510,158.051710,154.893332,214.531676,...,1138.472215,1078.403904,1032.597254,1000.837735,982.967875,2209.151602,2038.430094,1884.254564,1746.309413,1426.716895
3497,-0.412277,-0.454341,-0.418252,216.185622,202.980242,200.948823,142.465927,130.096997,128.199704,185.638124,...,1213.408476,1154.439336,1109.813588,1079.335050,1062.862652,2285.624036,2115.908406,1962.726095,1825.770357,1509.237715
3498,-1.054157,4.376763,-1.063629,492.132543,585.629402,694.312944,405.744502,497.030522,603.563898,154.449276,...,1132.583230,1193.576669,1269.163138,1359.630695,1465.291839,1315.481483,1277.507022,1253.856682,1244.434655,1301.441578


In [6]:
sample = 80
test_sample = 20
sto_ind = np.random.randint(0,len(mat_xs),sample)

In [7]:
sto_ind

array([3399, 1594, 3372, 2652,  980, 2030, 2193, 1139, 2412,  156, 2920,
        506,  629, 2226, 3272, 3014, 2598, 2578, 1227, 3407, 1194, 1477,
       1358,  341,   36, 1604, 1313,   60,  596,  761, 3355,   76, 2024,
       1279, 2992, 1917, 1229, 3417,  139, 3042,  610, 2970, 2715,  548,
       2494, 2032,  817, 2464, 2001, 1933,  851,  881, 3186, 2875, 2426,
       2792,  319, 1348,  268, 1591,  332, 3369, 3353,  534, 1223, 2604,
        575,  194, 2473,  206, 1948, 2574, 3244,  908, 2010, 1694,  725,
       3113, 2684,  568])

In [8]:
test_sto_ind = np.random.randint(0,len(mat_xs),test_sample)
test_sto_ind

array([ 759, 1783, 3121, 2812, 1527, 1216, 1789, 1842, 1454,  281,  954,
       2842, 3157, 2102, 1715, 2855, 2521,  359,  978,  682])

In [9]:
data_sto = mat_xs.iloc[sto_ind,:]
data_sto_y = mat_ys.iloc[sto_ind]

data_sto_test = mat_xs.iloc[test_sto_ind,:]
data_sto_y_test = mat_ys.iloc[test_sto_ind]

In [10]:
kernel_list = [kernel_H,kernel_I,kernel_J,kernel_K]

In [11]:
gram_kernel_set= []
gram_kernel_test_set =[]
comp=1
backend = QasmSimulator()

gram_matrix = Gram_gaussian(data_sto,comp)
gram_test_matrix = Gram_gaussian_test(data_sto,data_sto_test,comp)
    
gram_kernel_set += [gram_matrix]
gram_kernel_test_set += [gram_test_matrix]

for i in range(4) :
    gram_matrix = get_gram_multi(data=data_sto,kernel_fun=kernel_list[i],layer=1,backend=backend,shots=1000)
    gram_test_matrix = get_gram_test_multi(data=data_sto,test_data = data_sto_test,kernel_fun=kernel_list[i],layer=1,backend=backend,shots=1000)
    
    pd.DataFrame(gram_matrix)

    gram_kernel_set += [gram_matrix]
    gram_kernel_test_set += [gram_test_matrix]

/Users/minhyeong-gyu/Documents/GitHub/quantum_kernel_investigation/Quantum_Kernel.py:330: FutureWarning: Calling a ufunc on non-aligned DataFrames (or DataFrame/Series combination). Currently, the indices are ignored and the result takes the index/columns of the first DataFrame. In the future , the DataFrames/Series will be aligned before applying the ufunc.
Convert one of the arguments to a NumPy array (eg 'ufunc(df1, np.asarray(df2)') to keep the current behaviour, or align manually (eg 'df1, df2 = df1.align(df2)') before passing to the ufunc to obtain the future behaviour and silence this warning.
  U = np.matmul(data,data.T)
/Users/minhyeong-gyu/Documents/GitHub/quantum_kernel_investigation/Quantum_Kernel.py:344: FutureWarning: Calling a ufunc on non-aligned DataFrames (or DataFrame/Series combination). Currently, the indices are ignored and the result takes the index/columns of the first DataFrame. In the future , the DataFrames/Series will be aligned before applying the ufunc.
Co

KeyboardInterrupt: 

In [ ]:
for i in range(5):
    gram_kernel_set[i].to_csv("malaria2/gram_matrix"+str(i)+".csv")
    gram_kernel_test_set[i].to_csv("malaria2/gram_matrix_test"+str(i)+".csv")